In [2]:
import os
import glob
import pandas as pd

In [6]:
path =r'/Users/michaelpiacentino/Desktop/smpd3_dataset/smpd3_gRNA1/20171016/wm/migration/pax7migrationCSV/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label and Image name columns
    df['delete1'], df['ROI'], df['delete2'], df['delete3'] = zip(*df['Label'].map(lambda x: x.split(':')))
    df['Embryo'], df['Somites'], df['pic'] = zip(*df['Image'].map(lambda x: x.split('_')))
    
    # Replace ROI names
    df.replace(to_replace=['Cntl1', 'Cntl2', 'Cntl3', 'Cntl4', 'Exp1', 'Exp2', 'Exp3', 'Exp4'],
                            value=['Cntl', 'Cntl', 'Cntl', 'Cntl', 'Expt', 'Expt', 'Expt', 'Expt'], 
                            inplace=True)
    list_.append(df)

full_df = pd.concat(list_)
full_df = full_df.drop(['Label', 'Mean', 'Angle', 'IntDen', 'RawIntDen', 'pic', 'delete1', 'delete2', 'delete3'], axis=1)


In [7]:
# Mean length values and group
area_meanLength = (full_df.groupby(['Embryo', 'ROI'])['Area', 'Length']).mean()

In [8]:
# Extract Areas, generate ratios
area_cntl = area_meanLength.xs('CntlArea', level='ROI')['Area'] 
area_expt = area_meanLength.xs('ExptArea', level='ROI')['Area'] 

# Generate ratios as Expt/Cntl
area_ratios = pd.DataFrame(area_expt / area_cntl)
area_ratios.columns = ['Expt/Cntl Area']

In [9]:
# Extract lengths, generate ratios
length_cntl = area_meanLength.xs('Cntl', level='ROI')['Length'] 
length_expt = area_meanLength.xs('Expt', level='ROI')['Length'] 

# Generate ratios as Expt/Cntl
length_ratios = pd.DataFrame(length_expt / length_cntl)
length_ratios.columns = ['Expt/Cntl Length']

In [10]:
# Combine processed values into single dataframe and output as csv file 'Results.csv'
area_cntl = pd.DataFrame(area_cntl)
area_cntl.columns = ['Cntl Area']
area_expt = pd.DataFrame(area_expt)
area_expt.columns = ['Expt Area']

length_cntl = pd.DataFrame(length_cntl)
length_cntl.columns = ['Cntl Length']
length_expt = pd.DataFrame(length_expt)


results = pd.concat([area_cntl, area_expt, area_ratios, length_cntl, length_expt, length_ratios], axis=1, sort=True)
results.to_csv('Results.csv')